In [1]:
#library(lme4)
library(MCMCglmm)
library(broom)
library(nadiv)
library(tidyverse) #%>%

Warning message:
"package 'MCMCglmm' was built under R version 3.6.3"
Loading required package: Matrix

Warning message:
"package 'Matrix' was built under R version 3.6.3"
Loading required package: coda

Warning message:
"package 'coda' was built under R version 3.6.3"
Loading required package: ape

Warning message:
"package 'ape' was built under R version 3.6.3"
Warning message:
"package 'nadiv' was built under R version 3.6.3"

Attaching package: 'nadiv'


The following object is masked from 'package:MCMCglmm':

    prunePed


Warning message:
"package 'tidyverse' was built under R version 3.6.3"
-- Attaching packages --------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.1     v dplyr   1.0.6
v tidyr   1.1.3     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

Warning message:
"package 'tibble' was built under R version 3.6.3"
Warning message:
"package 'tidyr' was built under R version 3.6.3"
Warning message:
"pa

In [2]:
library(parallel)
my.cores = detectCores()

# Download data

In [3]:
path_adapt = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_/Adaptability'
df = read.csv(file.path(path_adapt,'df_MVT_4individuality_withPCA.csv'), header = TRUE, sep = ",")
df$HenID = as.factor(df$HenID)   
df$PenID = as.factor(df$PenID) 
#mean-centering of the environmental variable so that intercepts reflect average values for the population and individuals
df$cDIB = as.integer(df$DIB) 
df$cDIB2 = poly(df$cDIB, degree=2,raw=TRUE)[,2]
df$CLASS = as.factor(df$CLASS) 
df$Treatment = as.factor(df$Treatment) 
#scale all continuous fixed effectcs
df$temperature_C_avg_scale = scale(df$temperature_C_avg, center=TRUE, scale=TRUE)
df$cDIB2_scale = scale(df$cDIB2, center=TRUE, scale=TRUE)
df$cDIB_scale = scale(df$cDIB, center=TRUE, scale=TRUE)
df$avgDIB_scale = scale(df$avgDIB, center=TRUE, scale=TRUE)
df$InitialWeight_scale = scale(df$InitialWeight, center=TRUE, scale=TRUE)
print(dim(df))
summary(df)

[1] 3230   33


     HenID           DIB             CLASS          PenID     temperature_C_avg
 hen_116:  43   Min.   :11.00   EPI     : 478   pen10  :504   Min.   : 4.000   
 hen_124:  43   1st Qu.:23.00   LEXP    : 965   pen11  :504   1st Qu.: 8.667   
 hen_127:  43   Median :33.00   LEXPLOST: 366   pen8   :430   Median :11.000   
 hen_136:  43   Mean   :33.59   MEXP    :1379   pen9   :429   Mean   :10.838   
 hen_147:  43   3rd Qu.:44.00   MEXPLOST:  42   pen4   :388   3rd Qu.:12.667   
 hen_151:  43   Max.   :54.00                   pen5   :363   Max.   :19.333   
 (Other):2972                                   (Other):612                    
  tobeusedPCA     InitialWeight  Treatment_encoded    perc_duration_5_Zone
 Min.   :0.0000   Min.   :1046   Min.   :-1.0000000   Min.   :  0.00      
 1st Qu.:0.0000   1st Qu.:1130   1st Qu.:-1.0000000   1st Qu.: 11.00      
 Median :0.0000   Median :1164   Median :-1.0000000   Median : 33.00      
 Mean   :0.1734   Mean   :1169   Mean   :-0.0006192   Mean  

In [4]:
length(unique(df[['HenID']]))

[1] 80

In [5]:
nsamp = 4000
burn = 50000 #Matt: I would start with something more along 5000 no? is pattern i trace plot then augment
thin = 2000 #interval at which the Markov chain is stored
nitt = burn + nsamp * thin  #number of iterations

In [6]:
#FROM INTERNET!!!
clean.MCMC <- function(x) {
    sols <- summary(x)$solutions  ## pull out relevant info from model summary
    Gcovs <- summary(x)$Gcovariances
    Rcovs <- summary(x)$Rcovariances
    fixed <- data.frame(row.names(sols), sols, row.names = NULL)  ## convert to dataframes with the row.names as the first col
    random <- data.frame(row.names(Gcovs), Gcovs, row.names = NULL)
    residual <- data.frame(row.names(Rcovs), Rcovs, row.names = NULL)
    names(fixed)[names(fixed) == "row.names.sols."] <- "variable"  ## change the columns names to variable, so they all match
    names(random)[names(random) == "row.names.Gcovs."] <- "variable"
    names(residual)[names(residual) == "row.names.Rcovs."] <- "variable"
    fixed$effect <- "fixed"  ## add ID column for type of effect (fixed, random, residual)
    random$effect <- "random"
    residual$effect <- "residual"
    modelTerms <- as.data.frame(bind_rows(fixed, random, residual))  # merge it all together
}
getName.MCMC <- function(x) deparse(substitute(x))  # add the model name

# Random Intercept Model

In [ ]:
#with PenID
RI_prior = list(R = list(V = 1, nu = 1.002),
                G = list(G1 = list(V = 1, nu = 1.002, alpha.mu = 0, alpha.V = 1000),
                         G2 = list(V = 1, nu = 1.002, alpha.mu = 0, alpha.V = 1000)))
set.seed(123)
R1_model1 = MCMCglmm(PC1 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale + InitialWeight_scale,
                    random = ~ PenID + PenID:HenID,
                    prior = RI_prior,
                    data = df,
                    family = "gaussian",
                    pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE, thin = thin, burnin= burn, nitt= nitt)
plot(R1_model1$VCV)
#trace plots for fixed effect
plot(R1_model1$Sol)
summary(R1_model1)

####control long-term repeatability estimates with penid###
#new posterior distribution of repeatability: ‘proportion of exploration variance explained by differences among individuals
PCrepeat1 =  R1_model1$VCV[,"PenID:HenID"]/(R1_model1$VCV[,"PenID:HenID"] + R1_model1$VCV[,"units"])
mean(PCrepeat1)
posterior.mode(PCrepeat1)
HPDinterval(PCrepeat1)

#save
df_saved = clean.MCMC(R1_model1)  # get all the info from summary(modelName)
df_saved$modelName = getName.MCMC(R1_model1)  # add the model's name in a new column
head(df_saved,3)  # check out the created dataframe
write.csv(df_saved, file.path(path_adapt,'R1_model1.csv'))

In [ ]:
autocorr.diag(R1_model1$Sol[,1:9]) #8 fixed effect and intercept
autocorr.diag(R1_model1$VCV)

In [ ]:
#save
df_saved = clean.MCMC(R1_model1)  # get all the info from summary(modelName)
df_saved$modelName = getName.MCMC(R1_model1)  # add the model's name in a new column
head(df_saved,3)  # check out the created dataframe
write.csv(df_saved, file.path(path_adapt,'R1_model1.csv'))

In [ ]:
#without PenID
RI_prior = list(R = list(V = 1, nu = 1.002),
                G = list(G1 = list(V = 1, nu = 1.002, alpha.mu = 0, alpha.V = 1000)))
set.seed(123)
RI_model2 = MCMCglmm(PC1 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale + InitialWeight_scale,
                    random = ~ HenID,
                    prior = RI_prior,
                    data = df,
                    family = "gaussian",
                    pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE, thin = thin, burnin= burn, nitt= nitt)
plot(RI_model2$VCV)
#trace plots for fixed effect
plot(RI_model2$Sol)
summary(RI_model2)

####control long-term repeatability estimates without penid###
#new posterior distribution of repeatability: ‘proportion of exploration variance explained by differences among individuals
PCrepeat2 =  RI_model2$VCV[,"HenID"]/(RI_model2$VCV[,"HenID"] + RI_model2$VCV[,"units"])
mean(PCrepeat2)
posterior.mode(PCrepeat2)
HPDinterval(PCrepeat2)


In [ ]:
autocorr.diag(RI_model2$Sol[,1:9]) #8 fixed effect and intercept
autocorr.diag(RI_model2$VCV)

In [ ]:
#save
df_saved = clean.MCMC(RI_model2)  # get all the info from summary(modelName)
df_saved$modelName = getName.MCMC(RI_model2)  # add the model's name in a new column
head(df_saved,3)  # check out the created dataframe
write.csv(df_saved, file.path(path_adapt,'RI_model2.csv'))

# Linear Random Slope Model

In [ ]:
#with PenID
#prior
RR_prior = list(R = list(V = 1, nu = 1.002),
                G = list(G1 = list(V = 1, nu = 1.002, alpha.mu = 0, alpha.V = 1000),
                         G2 = list(V = diag(2), nu = 2, 
                                   alpha.mu = rep(0,2), 
                                   alpha.V = diag(2) * 25^2)))
#PC1
set.seed(123)
RR_model1 = MCMCglmm(PC1 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale + InitialWeight_scale,
                     random = ~ PenID + us(1 + cDIB_scale):PenID:HenID,
                     prior = RR_prior,
                     data = df,
                     family = "gaussian",
                     pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE,thin = thin, burnin= burn, nitt= nitt)
summary(RR_model1)


#posterior distribution of the among-individual variation in linear slopes term
plot(RR_model1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"])
mean(RR_model1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"])
HPDinterval(RR_model1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"])

#################################### correlations random effects & random effects
###correlation between intercept and linear slope###
corr_IL = RR_model1$VCV[,"cDIB_scale:(Intercept).PenID:HenID"]/
  (sqrt(RR_model1$VCV[,"(Intercept):(Intercept).PenID:HenID"])*
     sqrt(RR_model1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"]))
posterior.mode(corr_IL)
HPDinterval(corr_IL)


In [ ]:
autocorr.diag(RR_model1$Sol[,1:9]) #8 fixed effect and intercept
autocorr.diag(RR_model1$VCV)

In [ ]:

#save
df_saved = clean.MCMC(RR_model1)  # get all the info from summary(modelName)
df_saved$modelName = getName.MCMC(RR_model1)  # add the model's name in a new column
head(df_saved,3)  # check out the created dataframe
write.csv(df_saved, file.path(path_adapt,'RR_model1.csv'))

In [ ]:
#without PenID
#prior
RR_prior = list(R = list(V = 1, nu = 1.002),
                G = list(G1 = list(V = diag(2), nu = 2, 
                                   alpha.mu = rep(0,2), 
                                   alpha.V = diag(2) * 25^2)))
#PC1
set.seed(123)
RR_model2 = MCMCglmm(PC1 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale + InitialWeight_scale,
                     random = ~ us(1 + cDIB_scale):HenID,
                     prior = RR_prior,
                     data = df,
                     family = "gaussian",
                     pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE,thin = thin, burnin= burn, nitt= nitt)
summary(RR_model2)

#posterior distribution of the among-individual variation in linear slopes term
plot(RR_model2$VCV[,"cDIB_scale:cDIB_scale.HenID"])
mean(RR_model2$VCV[,"cDIB_scale:cDIB_scale.HenID"])
HPDinterval(RR_model2$VCV[,"cDIB_scale:cDIB_scale.HenID"])

#################################### correlations random effects & random effects
###correlation between intercept and linear slope###
corr_IL = RR_model2$VCV[,"cDIB_scale:(Intercept).HenID"]/
  (sqrt(RR_model2$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model2$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_IL)
HPDinterval(corr_IL)


In [ ]:
autocorr.diag(RR_model2$Sol[,1:9]) #8 fixed effect and intercept
autocorr.diag(RR_model2$VCV)

In [ ]:
#save
df_saved = clean.MCMC(RR_model2)  # get all the info from summary(modelName)
df_saved$modelName = getName.MCMC(RR_model2)  # add the model's name in a new column
head(df_saved,3)  # check out the created dataframe
write.csv(df_saved, file.path(path_adapt,'RR_model2.csv'))

# Quadratic Random Slope Model

In [ ]:
#with PenID
#prior
RR_prior = list(R = list(V = 1, nu = 1.002),
                G = list(G1 = list(V = 1, nu = 1.002, alpha.mu = 0, alpha.V = 1000),
                         G2 = list(V = diag(3), nu = 3, 
                                   alpha.mu = rep(0,3), 
                                   alpha.V = diag(3) * 25^2)))
#PC1
set.seed(123)
RR_model_Q1 = MCMCglmm(PC1 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale + InitialWeight_scale,
                     random = ~ PenID + us(1 + cDIB_scale + cDIB2_scale):PenID:HenID,
                     prior = RR_prior,
                     data = df,
                     family = "gaussian",
                     pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE,thin = thin, burnin= burn, nitt= nitt)
summary(RR_model_Q1)


#posterior distribution of the among-individual variation in linear slopes term
plot(RR_model_Q1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"])
mean(RR_model_Q1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"])
HPDinterval(RR_model_Q1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"])

#posterior distribution of the among-individual variation in quadratic slopes term
plot(RR_model_Q1$VCV[,"cDIB2_scale:cDIB2_scale.PenID:HenID"])
mean(RR_model_Q1$VCV[,"cDIB2_scale:cDIB2_scale.PenID:HenID"])
HPDinterval(RR_model_Q1$VCV[,"cDIB2_scale:cDIB2_scale.PenID:HenID"])

#################################### correlations random effects & random effects
###correlation between intercept and linear slope###
corr_IL = RR_model_Q1$VCV[,"cDIB_scale:(Intercept).PenID:HenID"]/
  (sqrt(RR_model_Q1$VCV[,"(Intercept):(Intercept).PenID:HenID"])*
     sqrt(RR_model_Q1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"]))
posterior.mode(corr_IL)
HPDinterval(corr_IL)

###correlation between intercept and quadratic linear slope###
corr_IQ = RR_model_Q1$VCV[,"cDIB2_scale:(Intercept).PenID:HenID"]/
  (sqrt(RR_model_Q1$VCV[,"(Intercept):(Intercept).PenID:HenID"])*
     sqrt(RR_model_Q1$VCV[,"cDIB2_scale:cDIB2_scale.PenID:HenID"]))
posterior.mode(corr_IQ)
HPDinterval(corr_IQ)

###correlation between linear slope and quadratic linear slope###
corr_LQ = RR_model_Q1$VCV[,"cDIB_scale:cDIB2_scale.PenID:HenID"]/
  (sqrt(RR_model_Q1$VCV[,"cDIB2_scale:cDIB2_scale.PenID:HenID"])*
     sqrt(RR_model_Q1$VCV[,"cDIB_scale:cDIB_scale.PenID:HenID"]))
posterior.mode(corr_LQ)
HPDinterval(corr_LQ)


In [ ]:
autocorr.diag(RR_model_Q1$Sol[,1:9]) #8 fixed effect and intercept
autocorr.diag(RR_model_Q1$VCV)

In [ ]:
#save
df_saved = clean.MCMC(RR_model_Q1)  # get all the info from summary(modelName)
df_saved$modelName = getName.MCMC(RR_model_Q1)  # add the model's name in a new column
head(df_saved,3)  # check out the created dataframe
write.csv(df_saved, file.path(path_adapt,'RR_model_Q1.csv'))

In [ ]:
#without PenID
#prior
RR_prior = list(R = list(V = 1, nu = 1.002),
                G = list(G1 = list(V = diag(3), nu = 3, 
                                   alpha.mu = rep(0,3), 
                                   alpha.V = diag(3) * 25^2)))
#PC1
set.seed(123)
RR_model_Q2 = MCMCglmm(PC1 ~ cDIB_scale + cDIB2_scale + avgDIB_scale + Treatment_encoded + CLASS + temperature_C_avg_scale + InitialWeight_scale,
                     random = ~ us(1 + cDIB_scale + cDIB2_scale):HenID,
                     prior = RR_prior,
                     data = df,
                     family = "gaussian",
                     pr = TRUE, verbose = FALSE, saveX = TRUE, saveZ = TRUE,thin = thin, burnin= burn, nitt= nitt)
summary(RR_model_Q2)

#posterior distribution of the among-individual variation in linear slopes term
plot(RR_model_Q2$VCV[,"cDIB_scale:cDIB_scale.HenID"])
mean(RR_model_Q2$VCV[,"cDIB_scale:cDIB_scale.HenID"])
HPDinterval(RR_model_Q2$VCV[,"cDIB_scale:cDIB_scale.HenID"])

#posterior distribution of the among-individual variation in quadratic slopes term
plot(RR_model_Q2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])
mean(RR_model_Q2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])
HPDinterval(RR_model_Q2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])

#################################### correlations random effects & random effects
###correlation between intercept and linear slope###
corr_IL = RR_model_Q2$VCV[,"cDIB_scale:(Intercept).HenID"]/
  (sqrt(RR_model_Q2$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model_Q2$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_IL)
HPDinterval(corr_IL)

###correlation between intercept and quadratic linear slope###
corr_IQ = RR_model_Q2$VCV[,"cDIB2_scale:(Intercept).HenID"]/
  (sqrt(RR_model_Q2$VCV[,"(Intercept):(Intercept).HenID"])*
     sqrt(RR_model_Q2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"]))
posterior.mode(corr_IQ)
HPDinterval(corr_IQ)

###correlation between linear slope and quadratic linear slope###
corr_LQ = RR_model_Q2$VCV[,"cDIB_scale:cDIB2_scale.HenID"]/
  (sqrt(RR_model_Q2$VCV[,"cDIB2_scale:cDIB2_scale.HenID"])*
     sqrt(RR_model_Q2$VCV[,"cDIB_scale:cDIB_scale.HenID"]))
posterior.mode(corr_LQ)
HPDinterval(corr_LQ)

In [ ]:
autocorr.diag(RR_model_Q2$Sol[,1:9]) #8 fixed effect and intercept
autocorr.diag(RR_model_Q2$VCV)

In [ ]:
#save
df_saved = clean.MCMC(RR_model_Q2)  # get all the info from summary(modelName)
df_saved$modelName = getName.MCMC(RR_model_Q2)  # add the model's name in a new column
head(df_saved,3)  # check out the created dataframe
write.csv(df_saved, file.path(path_adapt,'RR_model_Q2.csv'))

# Prediction of best model

In [ ]:
path_adapt = 'C:/Users/camil/Desktop/animals_code/PhD-AnimalWelfare/Chapter1-EarlyMovementLateHealth/FICHIER'
save(R1_model1, file = file.path(path_adapt,"R1_model1.RData"))
#load(coda.samples.RData)
path_adapt = 'C:/Users/camil/Desktop/animals_code/PhD-AnimalWelfare/Chapter1-EarlyMovementLateHealth/FICHIER'
save(RI_model2, file = file.path(path_adapt,"RI_model2.RData"))
#load(coda.samples.RData)
path_adapt = 'C:/Users/camil/Desktop/animals_code/PhD-AnimalWelfare/Chapter1-EarlyMovementLateHealth/FICHIER'
save(RR_model1, file = file.path(path_adapt,"RR_model1.RData"))
#load(coda.samples.RData)
path_adapt = 'C:/Users/camil/Desktop/animals_code/PhD-AnimalWelfare/Chapter1-EarlyMovementLateHealth/FICHIER'
save(RR_model2, file = file.path(path_adapt,"RR_model2.RData"))
#load(coda.samples.RData)
path_adapt = 'C:/Users/camil/Desktop/animals_code/PhD-AnimalWelfare/Chapter1-EarlyMovementLateHealth/FICHIER'
save(RR_model_Q1, file = file.path(path_adapt,"RR_model_Q1.RData"))
#load(coda.samples.RData)
path_adapt = 'C:/Users/camil/Desktop/animals_code/PhD-AnimalWelfare/Chapter1-EarlyMovementLateHealth/FICHIER'
save(RR_model_Q2, file = file.path(path_adapt,"RR_model_Q2.RData"))
#load(coda.samples.RData)

In [ ]:
model_test = load(file.path(path_adapt,"RR_model_Q2.RData"))

In [ ]:
model_test

In [ ]:
R1_model1$DIC
RI_model2$DIC
RR_model1$DIC
RR_model2$DIC
RR_model_Q1$DIC
RR_model_Q2$DIC

In [ ]:
#Predictions
df_rand = cbind(df,
                ri1_fit_marginalnull =predict(R1_model1, marginal = NULL), #TODO: what does marginal mean??
                ri2_fit_marginalnull =predict(RI_model2, marginal = NULL),
                rr1_fit_marginalnull =predict(RR_model1, marginal = NULL),
                rr2_fit_marginalnull =predict(RR_model2, marginal = NULL),
                rrq1_fit_marginalnull =predict(RR_model_Q1, marginal = NULL),
                rrq2_fit_marginalnull =predict(RR_model_Q2, marginal = NULL),
                ri1_fit_marginalnull =predict(R1_model1, marginal = ~PenID + PenID:HenID),
                ri2_fit_marginalnull =predict(RI_model2, marginal = ~HenID),
                rr1_fit_marginalnull =predict(RR_model1, marginal = ~PenID + us(1 + cDIB_scale):PenID:HenID),
                rr2_fit_marginalnull =predict(RR_model2, marginal = ~us(1 + cDIB_scale):HenID),
                rrq1_fit_marginalnull =predict(RR_model_Q1, marginal = ~PenID + us(1 + cDIB_scale + cDIB2_scale):PenID:HenID),
                rrq2_fit_marginalnull =predict(RR_model_Q2, marginal = ~us(1 + cDIB_scale + cDIB2_scale):HenID))

In [ ]:
head(df_rand,2)

In [ ]:
write.csv(df_rand,file.path(path_adapt,'df_MVT_4individuality_withPCA_withPredictions.csv'))